# IRAF-like source detection with simulated images

## Overview

IRAF provided a couple of ways, `starfind` and `daofind`, to detect *stellar* sources in an image. [photutils]() provides similar functionality via the [`DAOStarFinder`]() and [`IRAFStarFinder`]() objects

This notebook will focus on [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) to emphasize that these options work well for detecting *stars* or other objects with a stellar profile. They do not work well for more extended objects.

The next couple of notebooks will:

+ apply these techniques to the Hubble Extreme Deep Field, to illustrate the differences between [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) and [`IRAFStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.IRAFStarFinder.html#photutils.detection.IRAFStarFinder), and to introduce another method of source detection,
+ apply the techniques to an image of stars from a ground-based telescope, and
+ illustrate other source detection techniques that work well for extended sources

## Simulated image of galaxies and stars

In the first part of this notebook we consider an image that includes 100 sources, all with Gaussian profiles, some of which are fairly elongated. We used this image in the previous section about removing the background prior to source detection.

As usual, we begin with some imports.

In [ ]:
from astropy.stats import sigma_clipped_stats, gaussian_sigma_to_fwhm
from astropy.table import QTable
from astropy.visualization import simple_norm, SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

import matplotlib.pyplot as plt
import numpy as np

from photutils.aperture import CircularAperture, EllipticalAperture
from photutils.datasets import make_100gaussians_image, make_gaussian_sources_image
from photutils.detection import find_peaks, DAOStarFinder

plt.style.use('../photutils_notebook_style.mplstyle')

To begin, let's create and look the the image. We create an image normalization here that we will use for displaying the image throughout the notebook.

In [ ]:
data = make_100gaussians_image()
norm = ImageNormalize(stretch=SqrtStretch())
plt.figure()
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
plt.grid()

Note that some of these object are star-like -- the ones that are darkest are close to circular, for example. Others are very elongated and there are a variety of brightnesses among the objects.

### Estimate and subtract background 

The background must be subtracted from the image before the sources are detected. For simplicity we estimate the sigma-clipped mean, median and standard deviation of the pixels in the image. As discussed in the section on [background removal for source detection](), the sigma-clipped median gives a reasonable estimate of the background in many cases.

In [ ]:
mean, med, std = sigma_clipped_stats(data, sigma=3.0, maxiters=5)
data_subtracted = data - med

### Detect sources

The source detection itself is a couple of lines of code:

In [ ]:
daofind = DAOStarFinder(fwhm=5.0, threshold=5.*std) 
sources = daofind(data_subtracted)

There are a variety of options you can select, detailed in the documentation for [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder). We only use two for now, specifying the typical full-width half-max (FWHM) of a source, and the threshold above which to conider something a source.

A summary of the sources detected shows that [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) detect 41 sources, substantially fewer than the 100 that are in the image.

In [ ]:
# Format the columns to only display two decimal points 
for col in sources.colnames:  
    if col not in ('id', 'npix'):
        sources[col].info.format = '%.2f'  # for consistent table output
sources.pprint(max_width=76)  


A plot showing the location of all of the detected sources helps illustrate why some of sources were missed, To display where sources were detected, we create a a set of photutils circular apertures and use its plot method to add a circle over each detected source.

In [ ]:
# CircularAperture expects a particular format for input positions...
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
apertures = CircularAperture(positions, r=4.0)

plt.figure()
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='blue', lw=1.5, alpha=0.5);

It looks like the sources that were detected are those that were not too faint and not too elongaetd. 

Let's look at the properties of the sources in this image, and explore some of the settings in [`DAOStarFinder`](https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder) to see how you could tune its parameters to detect a different subset of the sources.

### Sources in the image

The sources in this image are generated using the code below, which was copied from the [photutils source code here](https://photutils.readthedocs.io/en/stable/_modules/photutils/datasets/make.html#make_100gaussians_image).

The sources have a range of orientations, ellipticities, and widths. All of the sources are 2D Gaussians, with a FWHM ranging from roughly 3 to 15 pixels in each direction. The ratio of minor to major axis of the sources varies from 0.2 to 1.0.

In [ ]:
n_sources = 100
flux_range = [500, 1000]
xmean_range = [0, 500]
ymean_range = [0, 300]
xstddev_range = [1, 5]
ystddev_range = [1, 5]
params = {'flux': flux_range,
          'x_mean': xmean_range,
          'y_mean': ymean_range,
          'x_stddev': xstddev_range,
          'y_stddev': ystddev_range,
          'theta': [0, 2 * np.pi]}

rng = np.random.RandomState(12345)
inp_sources = QTable()
for param_name, (lower, upper) in params.items():
    # Generate a column for every item in param_ranges, even if it
    # is not in the model (e.g., flux).  However, such columns will
    # be ignored when rendering the image.
    inp_sources[param_name] = rng.uniform(lower, upper, n_sources)
xstd = inp_sources['x_stddev']
ystd = inp_sources['y_stddev']
inp_sources['amplitude'] = inp_sources['flux'] / (2.0 * np.pi * xstd * ystd)

Let's look at the image again, with a marker around each input source. There is quite a bit of information packed into those markers:

+ the shape of the marker matches the minor-to-major axis ratio
+ the major axis of the marker is related to the FWHM of the major axis
+ the thickness indicates the total flux of the source.


In [ ]:
plt.figure(figsize=(20, 10))

# plot the image
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')

# For each input source make an aperture for it
idx = 0
for source in inp_sources:
    minor = min(source['x_stddev'], source['y_stddev'])
    major = max(source['x_stddev'], source['y_stddev'])
    ratio = minor / major

    major_fwhm = major * gaussian_sigma_to_fwhm

    # ap = EllipticalAperture((source['x_mean'], source['y_mean']), major, minor, theta=source['theta'])
    ap = EllipticalAperture((source['x_mean'], source['y_mean']), source['x_stddev'], source['y_stddev'], theta=source['theta'])
    
    # This should give linewidths of 1, 2, or 3 given the input fluxes
    line_width = source['flux'] // 250 - 1
    ap.plot(color='blue', lw=line_width)

plt.title("Input sources")
    # plt.annotate(str(idx), (source['x_mean'] + 5, source['y_mean'] + 5))
    # idx += 1
    # plt.annotate(
    #     f"{minor / major:.2f}", 
    #     #f"{1 - source['x_stddev'] / source['y_stddev']:.2f}", 
    #     (source['x_mean'] + 5, source['y_mean'] + 5)
    # )
    # plt.annotate(
    #     f"{(source['x_stddev'] + source['x_stddev'])/2 * gaussian_sigma_to_fwhm:.2f}", 
    #     (source['x_mean'] + 5, source['y_mean'] - 5)
    # )
    # plt.annotate(
    #     f"{source['amplitude']:.2f}", 
    #     (source['x_mean'] + 5, source['y_mean'] - 5)
    # )


In [ ]:
daofind_source = DAOStarFinder(fwhm=5.0, threshold=5.*std, xycoords=inp_positions)  
sources_source = daofind_source(data - median)

In [ ]:
sources_source

## `find_peaks`

In [ ]:
threshold = median + (5.0 * std)
tbl = find_peaks(data, threshold, box_size=11)
tbl['peak_value'].info.format = '%.8g'  # for consistent table output
tbl.pprint(max_width=76)  

In [ ]:
positions = np.transpose((tbl['x_peak'], tbl['y_peak']))
apertures = CircularAperture(positions, r=5.0)
#norm = simple_norm(data, 'sqrt', percent=99.9)
plt.imshow(data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
apertures.plot(color='#0547f9', lw=1.5)
plt.xlim(0, data.shape[1] - 1)
plt.ylim(0, data.shape[0] - 1)

## Sources all theta=0

In [ ]:
flat_inp_sources = inp_sources.copy()
flat_inp_sources['theta'] = 0.0
minor_x = flat_inp_sources['x_stddev'] < flat_inp_sources['y_stddev']
majors = flat_inp_sources['x_stddev'].copy()
majors[minor_x] = flat_inp_sources['y_stddev'][minor_x]

minors = flat_inp_sources['y_stddev'].copy()
minors[minor_x] = flat_inp_sources['x_stddev'][minor_x]

flat_inp_sources['x_stddev'] = majors
flat_inp_sources['y_stddev'] = minors

flat_inp_sources['y_stddev'] = flat_inp_sources['x_stddev']

In [ ]:
shape = (300, 500)
flat_data = make_gaussian_sources_image(shape, flat_inp_sources) + 5.0

if True:
    rng = np.random.RandomState(12345)
    flat_data += rng.normal(loc=0.0, scale=2.0, size=shape)



In [ ]:
plt.figure(figsize=(20, 10))

plt.imshow(flat_data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')

In [ ]:
daofind = DAOStarFinder(fwhm=10.0, threshold=5.*std, ratio=1, sharplo=0.2)  
flat_sources = daofind(flat_data - median) 

In [ ]:
plt.figure(figsize=(20, 10))

flat_positions = np.transpose((flat_sources['xcentroid'], flat_sources['ycentroid']))
flat_apertures = CircularAperture(flat_positions, r=4.0)

plt.imshow(flat_data, cmap='Greys', origin='lower', norm=norm,
           interpolation='nearest')
flat_apertures.plot(color='blue', lw=1.5, alpha=0.5);
for source in flat_inp_sources:
    minor = min(source['x_stddev'], source['y_stddev'])
    major = max(source['x_stddev'], source['y_stddev'])
    plt.annotate(
        f"{minor / major:.2f}", 
        #f"{1 - source['x_stddev'] / source['y_stddev']:.2f}", 
        (source['x_mean'] + 5, source['y_mean'] + 5)
    )
    plt.annotate(
        f"{source['amplitude']:.2f}", 
        (source['x_mean'] + 5, source['y_mean'] - 5)
    )
    plt.annotate(
        f"{(source['x_stddev'] + source['x_stddev'])/2 * gaussian_sigma_to_fwhm:.2f}", 
        (source['x_mean'] + 5, source['y_mean'])
    )

In [ ]:
flat_sources.colnames